# 10. Design Patterns with First-Class Functions

In [30]:
from abc import ABC, abstractmethod 
from collections.abc import Sequence 
from decimal import Decimal
from typing import NamedTuple, Optional

In [31]:
from dis import disco


class Customer(NamedTuple):
    name: str
    fidelity: int

class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity

class Order(NamedTuple): # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))
    
    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'
    
class Promotion(ABC): # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""

class FidelityPromo(Promotion): # first concrete strategy
    """5% discount for customer with 1000 o more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)

class BulkItemPromo(Promotion): # second concrete stategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.10')
        return discount
    
class LargeOrderPromo(Promotion): # third concrete strategy
    """7% discount for orders with 10 or more distinc items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [32]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [33]:
cart = (
    LineItem('banana', 4, Decimal('0.5')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5))
        )

In [34]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [35]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [36]:
banana_cart = (
    LineItem('banana', 30, Decimal('0.50')),
    LineItem('apple', 10, Decimal('1.5'))
)

In [37]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [38]:
long_cart = tuple(LineItem(str(sku), 1, Decimal(1))
                  for sku in range(10)
                  )
Order(joe, long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [39]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## Function-Oriented Strategy

Each concrete strategy in example before is a class with a single method, `discount`. Furthermore, the strategy instances have no state (no instance attributes). You could say they look a lot like plain functions, and you would be right. Refactoring in example next, replacing the concrete strategies with simple functions and removing the `Promo` abstract class. Only small adjustments are needed in the `Order` class.

In [40]:
from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import Optional, Callable, NamedTuple

In [43]:
class Customer(NamedTuple):
    name: str
    fidelity: int

class LineItem(NamedTuple): 
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity

@dataclass(frozen=True) 
class Order: # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional[Callable[['Order'], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))
    
    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'
    
def fidelity_promo(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points""" 
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)

def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.10')
    return discount

def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items""" 
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07') 
    return Decimal(0)


In [45]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, Decimal('0.50')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5)),
]


In [46]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [48]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [49]:
banana_cart = [
    LineItem('banana', 30, Decimal('0.50')),
    LineItem('apple', 10, Decimal('1.5')),
]

In [50]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [51]:
long_cart = [
    LineItem(str(item_code), 1, Decimal(1))
    for item_code in range(10)
]

In [52]:
Order(joe, long_cart, large_order_promo)

<Order total: 10.00 due: 9.30>

## Finding Strategies in a Module

In [53]:
from decimal import Decimal
# from strategy import Order

promos = [ promo for name, promo in globals().items()
          if name.endswith('_promo') and name!='best_promo'
          ]

def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)


## Decorator-Enhanced Strategy Patter

In [54]:
Promotion = Callable[[Order], Decimal]
promos: list[Promotion] = []

def promotion(promo: Promotion) -> Promotion: 
    promos.append(promo)
    return promo

def best_promo(order: Order) -> Decimal: 
    """Compute the best discount available""" 
    return max(promo(order) for promo in promos)

@promotion
def fidelity(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points""" 
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05') 
    return Decimal(0)

@promotion
def bulk_item(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units""" 
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount

@promotion
def large_order(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items""" 
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07') 
    return Decimal(0)